In [ ]:
#!/usr/local/anaconda3/bin/pip install community

import pandas as pd
import numpy as np
import scipy.stats as st
import statistics
from pathlib import Path

import requests
import re
import os

from multiprocess import Pool
import psutil
from tqdm import tqdm

from bs4 import BeautifulSoup

import glob

import matplotlib.pyplot as plt

In [ ]:
#base_df = pd.read_csv("/data/schwurbelarchiv/extracted_information/dataframes/base_df.csv.gzip", compression = 'gzip')

In [ ]:
groups = base_df["group_name"].dropna().unique().tolist()

In [ ]:
import random

random.seed(42)
random_groups = random.sample(groups, 200)

sampled_df = text_df[text_df["group_name"].isin(random_groups)].reset_index()

doc_list = sampled_df["messages"].to_list()
date_list = sampled_df["day"].to_list()

In [ ]:
import nltk
nltk.download('stopwords')
from bertopic import BERTopic
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN

german_stop_words = stopwords.words('german')

vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words= german_stop_words)
umap_model = UMAP(n_neighbors=10, n_components =2, metric="cosine", min_dist=0.0)
hdbscan_model = HDBSCAN(min_cluster_size=5, metric="euclidean", prediction_data=True)


In [ ]:
topic_model = BERTopic(language="german",
                       vectorizer_model=vectorizer_model,
                       hdbscan_model=hdbscan_model,
                       umap_model=umap_model,
                       calculate_probabilities=False,
                       verbose=True)

In [ ]:
topics = topic_model.fit_transform(doc_list)

In [ ]:
hierarchical_topics = topic_model.hierarchical_topics(doc_list)
vis_topics = topic_model.visualize_hierarchical_documents(doc_list, hierarchical_topics)

In [ ]:
topics_over_time = topic_model.topics_over_time(doc_list, date_list, nr_bins=20)

In [ ]:
topic_model.visualize_topics_over_time(topics_over_time, top_n_topics=20)

In [ ]:
vis_topics

In [ ]:
topic_model.get_topic_info().head()

In [ ]:
#group_topics shows all topics of one group
sampled_df["topic"] = topics[0]

group_topics = sampled_df.groupby("group_name")["topic"].apply(list).reset_index()

In [ ]:
def get_top(some_list):
    return len(list(set(some_list)))

In [ ]:
group_topics["nr-topics"] = group_topics["topic"].apply(get_top)

In [ ]:
group_topics["topic_ratio"].mean()

In [ ]:
group_topics["topic_ratio"] = group_topics["nr-topics"]/group_topics["days"]

In [ ]:
overall_topic_ratio = group_topics["nr-topics"].sum()/group_topics["days"].sum()
overall_topic_ratio